In [1]:
# imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# allows to output plots in the notebook
%matplotlib inline

In [2]:
# changing to correct directory
# DO NOT RUN A SECOND TIME (Restart & Run All is allowed)
os.chdir("..")

## Figuring out when is electricity the cheapest
 - Current dateTime format is CET (UTC+1).
 - Estonia has UTC+2 in the winter and UTC+3 in the summer
 - Finding the right hours
    - This can be done most easily by grouping by hour and then checking which hours have the average lowest prices
    - Also a bit more advanced would be to convert it into percentages and then find it by percentages (100% being the daily highest and 0 being 0%) This is done like so:
        - First find the maximum price in a day
        - Second divide the hourly price with the daily maximum and round to about 2 zeros
    - Both approaches should give approx the same answer

In [3]:
# reading in files and initial cleaning
nordpool = pd.read_csv(os.path.join("data", "processed", "nordpool_estonia.csv"))
nordpool.dropna(inplace=True)
nordpool.drop_duplicates(inplace=True)

In [4]:
# data engineering
df_grouped_date = nordpool.groupby(by="Date").max()["elspot_price"].reset_index()
df_grouped_date.rename(columns={"elspot_price": "max_price"}, inplace=True)
nordpool = nordpool.merge(df_grouped_date, how="left", on="Date", copy=False)
nordpool["percentage_price"] = round((nordpool["elspot_price"] / nordpool["max_price"]) * 100, 2)
nordpool.drop(columns="max_price", inplace=True)
hourly_average_prices = nordpool.groupby(by="Time").mean()

In [5]:
# just cool visualizing functions
def display_two_axis_bar_chart(df, y1_name:str, y2_name:str, title:str):
    fig = go.Figure(
        data=[
            go.Bar(name=y1_name, x=df.index, y=df[y1_name], yaxis='y', offsetgroup=1),
            go.Bar(name=y2_name, x=df.index, y=df[y2_name], yaxis='y2', offsetgroup=2)
        ],
        layout={
            'yaxis': {'title': f'{y1_name} axis'},
            'yaxis2': {'title': f'{y2_name} axis', 'overlaying': 'y', 'side': 'right'}
        },
    )
    # Change the bar mode
    fig.update_layout(barmode='group', title=title)
    fig.show()

def create_bar_chart(df, y):
    return go.Bar(x=df.index, y=df[y])

In [9]:
# creating a dataframe sorted by elspot price
hourly_elspot_prices = hourly_average_prices.sort_values(by="elspot_price", ascending=True)

# creating a dataframe sorted by percentage price
hourly_percentage_prices = hourly_average_prices.sort_values(by="percentage_price", ascending=True)

# Viewing hourly average prices and the cheapest among them
display_two_axis_bar_chart(hourly_average_prices, 'elspot_price', 'percentage_price', 'Elspot price VS percentage price')

fig = make_subplots(rows=2, cols=2,
                    subplot_titles = ('Elspot price (6 hours)', 'Elspot price (8 hours)', 'Percentage price (6 hours)', 'Percentage price (8 hours)'))
lowest=6
for i in range(2):
    fig.add_trace(create_bar_chart(hourly_elspot_prices[:lowest],"elspot_price"), 1, i+1)
    fig.add_trace(create_bar_chart(hourly_percentage_prices[:lowest], "percentage_price"), 2, i+1)
    lowest=8
fig.show()

## Elspot price and percentage price give us almost the same answers


In [7]:
# Viewing hourly average percentages of prices and the lowest among them

## So what is the best time to use your electricity?
 - By hourly mean price (UTC+1)
    - The most expensive price can be 2 times larger than the cheapest
    - Cheapest 6 hours are 23:00-05:00
        - Estonian winter time: 00:00-06:00
        - Estonian summer time: 01:00-07:00
    - Cheapest 8 hours are 22:00-06:00
        - Estonian winter time: 23:00-07:00
        - Estonian summer time: 00:00-08:00